In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
from case.code_generator import generate, case_transformer

ImportError: cannot import name 'case_transformer' from 'case.code_generator' (/Users/clement/Developer/ai/tx-generator/case/code_generator.py)

In [ ]:
generate().invoke("Send 10 USDC to Alice")

In [ ]:
generate().invoke("Supply 100 USDC to Compound")

In [ ]:
from utils.printer import print_markdown

response = case_transformer().invoke(
    "How many steps for the tx: Swap 10 USDC to DAI on Uniswap"
)
print_markdown(response.content)

In [ ]:
response = case_transformer().invoke(
    "How many steps for the tx: Supply 100 USDC to Compound"
)
print_markdown(response.content)

In [ ]:
response = case_transformer().invoke("How many steps for the tx: Stake ETH to Lido.")
print_markdown(response.content)

In [ ]:
response = case_transformer().invoke("How many steps for the tx: Unstake from Lido.")
print_markdown(response.content)

In [ ]:
from utils.printer import print_markdown

response = case_transformer().invoke("How many steps for the tx: Supply USDT to AAVE")
print_markdown(response.content)

In [ ]:
from utils.printer import print_markdown

response = case_transformer().invoke(
    "How many steps for the tx: Orbit lending protocol case"
)
print_markdown(response.content)

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, MessageGraph
from langgraph.graph.message import add_messages


class CodeGenState(TypedDict):
    messages: Annotated[list, add_messages]
    has_error: bool


def chatbot(state: CodeGenState):
    return {"messages": generate(state["messages"])}


graph_builder = StateGraph(CodeGenState)
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()